# REMIND Electricity Sector Biosphere Flows

Calculate all biosphere flow coefficients for the REMIND electricity sector. **Values are given per kWh**.

In [53]:
from progressbar import progressbar
import helpers.eimod as eimod
import helpers.ei2rmnd as ei2rmnd

In [54]:
%run initialize_notebook.ipynb

In [55]:
databases

Databases dictionary with 10 object(s):
	Carma CCS
	Inventory flows
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

Add some inventory material flows to the biosphere:

In [56]:
ei2rmnd.techno_filters

{'steel': {'fltr': 'market for steel,', 'mask': 'hot rolled'},
 'concrete': {'fltr': 'market for concrete,'},
 'copper': {'fltr': 'market for copper', 'filter_exact': True},
 'aluminium': {'fltr': ['market for aluminium, primary',
   'market for aluminium alloy,']},
 'electricity': {'fltr': 'market for electricity'},
 'gas': {'fltr': 'market for natural gas,', 'mask': ['network', 'burned']},
 'diesel': {'fltr': 'market for diesel', 'mask': ['burned', 'electric']},
 'petrol': {'fltr': 'market for petrol,', 'mask': 'burned'},
 'freight': {'fltr': 'market for transport, freight'},
 'cement': {'fltr': 'market for cement,'},
 'heat': {'fltr': 'market for heat,'}}

In [ ]:
ei2rmnd.add_REMIND_technosphere_flows()

## Bioflows without double counting

Calculate impact for an electricity generating technology in the REMIND power sector excluding *other* activities that are part of the electricity sector to avoid double counting.

In [4]:
def rmnd_bioflows(region, scenario="BAU", year=2030, tech_primers={}, double_counting=False):
    """ Extract bioflows for the REMIND electricity sector in a REMIND region.
    
    In this version, for every REMIND technology, we exclude all other technologies
    when performing the LCA to avoid double counting.
        
    A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
    REMIND technology x.
    """
    eidb_name = ei2rmnd.get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)

    result = {}
    bio_names = []
    regions = ei2rmnd.ei_locations_in_remind_region(region)
    print("Found following regions for {}: {}".format(region, regions))
    
    actvts_by_tech = {}
    
    # populate activity dictionary
    print("Collecting ecoinvent activities:")
    for tech, act_list in progressbar(
        eimod.available_electricity_generating_technologies.items(), 
        prefix="Collecting ecoinvent activities: "):
        
        actvts_by_tech[tech] = [act 
                                for act_name in act_list 
                                for act in ei2rmnd.find_activities_by_name(act_name, eidb)]
    
    # flat activity list (needed for excludes)
    all_actvts = [
        act 
        for act_lst in actvts_by_tech.values()
        for act in act_lst] 

    for tech, all_tech_actvts in actvts_by_tech.items():
        print("Processing RMND tech `{}`".format(tech))

        # region specific techs that are adressed
        actvts = [act for act in all_tech_actvts if act["location"] in regions]
        
        if len(actvts) == 0:
            print("No activities found for {}".format(tech))
            continue
        
        demand = 1.
        
        # CHP plants are a problem, not only because they provide energy in megajoule units
        # find out which part is electric energy
        if "CHP" in tech:
            demand = 1./3.6
            
        # normal multi-region and multi-tech lca
        lca = ei2rmnd.multi_lca_average(actvts, demand)
        
        if not double_counting:
            # ... and remove the double-counting
            lca = ei2rmnd.remove_double_counting(lca, actvts, all_actvts)
        
        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        # flows are aggregated by the resp. material
        # we do not care *where* the material flows to
        result[tech] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [5]:
df = rmnd_bioflows("EUR")

N/A% (0 of 20) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']


100% (20 of 20) |########################| Elapsed Time: 0:00:47 Time:  0:00:47


Processing RMND tech `Biomass IGCC CCS`
Bionames changed, old length: 0
new length: 2090
Processing RMND tech `Biomass IGCC`
Processing RMND tech `Coal IGCC`
Processing RMND tech `Coal IGCC CCS`
Processing RMND tech `Coal PC CCS`
Processing RMND tech `Gas CCS`
Processing RMND tech `Biomass CHP`
Processing RMND tech `Coal PC`
Processing RMND tech `Coal CHP`
Processing RMND tech `Gas OC`
Processing RMND tech `Gas CC`
Processing RMND tech `Gas CHP`
Processing RMND tech `Geothermal`
Processing RMND tech `Hydro`
Processing RMND tech `Hydrogen`
No activities found for Hydrogen
Processing RMND tech `Nuclear`
Processing RMND tech `Oil`
Processing RMND tech `Solar CSP`
Processing RMND tech `Solar PV`
Processing RMND tech `Wind`


In [6]:
df = df.reset_index().rename({"level_0": "RMND Tech"}, axis=1).set_index(["RMND Tech", "flow"])
df.sample(20)

,,amount
RMND Tech,flow,
Biomass CHP,"Gold, Au 6.7E-4%, in ore, in ground",8.612102e-12
Gas CCS,copper,1.004464e-05
Coal CHP,Ethylene glycol monoethyl ether,0.000000e+00
Solar CSP,"Transformation, to forest, unspecified",1.143939e-06
Wind,"Pt, Pt 4.8E-4%, Pd 2.0E-4%, Rh 2.4E-5%, Ni 3.7E-2%, Cu 5.2E-2% in ore, in ground",6.215289e-11
Biomass IGCC CCS,Methyl borate,2.474690e-12
Biomass CHP,Radium-226,4.347731e-06
Geothermal,"Pt, Pt 4.8E-4%, Pd 2.0E-4%, Rh 2.4E-5%, Ni 3.7E-2%, Cu 5.2E-2% in ore, in ground",1.037271e-11
Oil,Diflubenzuron,1.066889e-10


## Bioflows from the electricity sector with double counting

E.g., flows from wind power plants include coal-generated electricity impacts that are already being accounted for by a `electricity production, coal` lca.

In [7]:
df_dc = rmnd_bioflows("EUR", double_counting=True)

N/A% (0 of 20) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']


100% (20 of 20) |########################| Elapsed Time: 0:00:53 Time:  0:00:53


Processing RMND tech `Biomass IGCC CCS`
Bionames changed, old length: 0
new length: 2090
Processing RMND tech `Biomass IGCC`
Processing RMND tech `Coal IGCC`
Processing RMND tech `Coal IGCC CCS`
Processing RMND tech `Coal PC CCS`
Processing RMND tech `Gas CCS`
Processing RMND tech `Biomass CHP`
Processing RMND tech `Coal PC`
Processing RMND tech `Coal CHP`
Processing RMND tech `Gas OC`
Processing RMND tech `Gas CC`
Processing RMND tech `Gas CHP`
Processing RMND tech `Geothermal`
Processing RMND tech `Hydro`
Processing RMND tech `Hydrogen`
No activities found for Hydrogen
Processing RMND tech `Nuclear`
Processing RMND tech `Oil`
Processing RMND tech `Solar CSP`
Processing RMND tech `Solar PV`
Processing RMND tech `Wind`


In [8]:
df_dc = df_dc.reset_index().rename({"level_0": "RMND Tech"}, axis=1).set_index(["RMND Tech", "flow"])
df_dc.sample(20)

,,amount
RMND Tech,flow,
Gas CC,Phosphorus trichloride,4.370198e-15
Hydro,Chlorpyrifos,1.555585e-13
Biomass CHP,"Molybdenum, 0.016% in sulfide, Mo 8.2E-3% and Cu 0.27% in crude ore, in ground",9.621808e-09
Coal IGCC,Acrylonitrile,0.000000e+00
Coal PC CCS,Diethanolamine,7.339356e-12
Gas OC,Propargite,0.000000e+00
Coal IGCC,"Iodine, 0.03% in water",1.110752e-10
Coal PC CCS,Flurochloridone,0.000000e+00
Coal CHP,Pyraclostrobin,5.599559e-14


## Impact of double counting

Compare versions with and without double counting.

In [42]:
fulldf = df.join(df_dc, lsuffix="_nodc", rsuffix="_dc")

In [43]:
fulldf["rel. (nodc/dc)"] = fulldf["amount_nodc"]/fulldf["amount_dc"]

In [44]:
fulldf["abs. (dc - nodc)"] = fulldf["amount_dc"] - fulldf["amount_nodc"]

In [45]:
fulldf.loc[fulldf["rel. (nodc/dc)"] > 0].sort_values("rel. (nodc/dc)", ascending=False)[:30]

amount_nodc  \
RMND Tech        flow                                                               
Biomass IGCC     Transformation, to dump site, residual material... -2.748097e-08   
                 Transformation, from dump site, residual materi... -2.748390e-08   
Coal IGCC CCS    Transformation, to dump site, inert material la... -2.168726e-08   
                 Transformation, from dump site, inert material ... -2.168726e-08   
Coal IGCC        Transformation, to dump site, sanitary landfill    -2.246612e-09   
                 Transformation, from dump site, sanitary landfill  -2.246612e-09   
Coal IGCC CCS    Cyanide                                            -4.627897e-09   
Biomass IGCC CCS Beryllium                                          -1.123339e-07   
Gas CCS          Antimony                                            1.717611e-08   
Coal PC CCS      Iodide                                             -4.837398e-09   
Coal IGCC CCS    Transformation, from pasture, man made             -3.683609e-06   
Coal IGCC        Cyanide                                            -4.862920e-09   
Coal IGCC CCS    Tungsten                                           -2.824768e-08   
Biomass IGCC CCS Lead                                               -3.397416e-07   
                 Manganese                                          -1.047758e-04   
                 Calcium, ion                                       -2.650006e-03   
                 Aluminium                                          -1.982630e-04   
Coal PC CCS      Transformation, to dump site, residual material... -3.421233e-08   
                 Transformation, from dump site, residual materi... -3.421361e-08   
Coal IGCC        Transformation, from pasture, man made             -2.883017e-06   
Biomass IGCC     Carbon                                             -7.735508e-06   
Biomass IGCC CCS Titanium, ion                                      -6.089228e-06   
Biomass IGCC     Beryllium                                          -3.563904e-07   
Coal IGCC        Tungsten                                           -2.646483e-08   
Biomass IGCC CCS Hydrogen sulfide                                   -6.966395e-06   
                 Transformation, to dump site, slag compartment     -1.164890e-08   
                 Transformation, from dump site, slag compartment   -1.164890e-08   
                 Carbon                                             -6.152168e-06   
Gas OC           Antimony                                            2.315695e-08   
Nuclear          Antimony                                            4.393911e-08   

                                                                        amount_dc  \
RMND Tech        flow                                                               
Biomass IGCC     Transformation, to dump site, residual material... -1.814532e-11   
                 Transformation, from dump site, residual materi... -1.259180e-10   
Coal IGCC CCS    Transformation, to dump site, inert material la... -6.761086e-09   
                 Transformation, from dump site, inert material ... -6.761086e-09   
Coal IGCC        Transformation, to dump site, sanitary landfill    -7.438979e-10   
                 Transformation, from dump site, sanitary landfill  -7.438979e-10   
Coal IGCC CCS    Cyanide                                            -2.237837e-09   
Biomass IGCC CCS Beryllium                                          -6.279383e-08   
Gas CCS          Antimony                                            1.053042e-08   
Coal PC CCS      Iodide                                             -3.231228e-09   
Coal IGCC CCS    Transformation, from pasture, man made             -2.463695e-06   
Coal IGCC        Cyanide                                            -3.306793e-09   
Coal IGCC CCS    Tungsten                                           -1.943343e-08   
Biomass IGCC CCS Lead                                               -2.350

In [46]:
fulldf.loc[fulldf["rel. (nodc/dc)"] > 1].sort_values(by="rel. (nodc/dc)", ascending=False)

amount_nodc  \
RMND Tech        flow                                                               
Biomass IGCC     Transformation, to dump site, residual material... -2.748097e-08   
                 Transformation, from dump site, residual materi... -2.748390e-08   
Coal IGCC CCS    Transformation, from dump site, inert material ... -2.168726e-08   
                 Transformation, to dump site, inert material la... -2.168726e-08   
Coal IGCC        Transformation, to dump site, sanitary landfill    -2.246612e-09   
                 Transformation, from dump site, sanitary landfill  -2.246612e-09   
Coal IGCC CCS    Cyanide                                            -4.627897e-09   
Biomass IGCC CCS Beryllium                                          -1.123339e-07   
Gas CCS          Antimony                                            1.717611e-08   
Coal PC CCS      Iodide                                             -4.837398e-09   
Coal IGCC CCS    Transformation, from pasture, man made             -3.683609e-06   
Coal IGCC        Cyanide                                            -4.862920e-09   
Coal IGCC CCS    Tungsten                                           -2.824768e-08   
Biomass IGCC CCS Lead                                               -3.397416e-07   
                 Manganese                                          -1.047758e-04   
                 Calcium, ion                                       -2.650006e-03   
                 Aluminium                                          -1.982630e-04   
Coal PC CCS      Transformation, to dump site, residual material... -3.421233e-08   
                 Transformation, from dump site, residual materi... -3.421361e-08   
Coal IGCC        Transformation, from pasture, man made             -2.883017e-06   
Biomass IGCC     Carbon                                             -7.735508e-06   
Biomass IGCC CCS Titanium, ion                                      -6.089228e-06   
Biomass IGCC     Beryllium                                          -3.563904e-07   
Coal IGCC        Tungsten                                           -2.646483e-08   
Biomass IGCC CCS Hydrogen sulfide                                   -6.966395e-06   
                 Transformation, to dump site, slag compartment     -1.164890e-08   
                 Transformation, from dump site, slag compartment   -1.164890e-08   
                 Carbon                                             -6.152168e-06   
Gas OC           Antimony                                            2.315695e-08   
Nuclear          Antimony                                            4.393911e-08   
...                                                                           ...   
Coal IGCC CCS    Indeno(1,2,3-cd)pyrene                             -3.121697e-15   
Coal IGCC        Indeno(1,2,3-cd)pyrene                             -2.609186e-15   
Biomass IGCC CCS Calcium                                            -2.189738e-04   
Coal IGCC        Benzo(ghi)perylene                                 -8.751522e-16   
Coal IGCC CCS    Benzo(ghi)perylene                                 -1.068103e-15   
Coal IGCC        Benzo(b)fluoranthene                               -1.422851e-14   
                 Benzo(k)fluoranthene                               -1.029338e-14   
                 Dibenz(a,h)anthracene                              -6.692181e-15   
Biomass IGCC CCS Phosphorus                                         -7.056733e-06   
Coal IGCC CCS    Benzo(b)fluoranthene                               -1.743734e-14   
                 Benzo(k)fluoranthene                               -1.262884e-14   
                 Dibenz(a,h)anthracene                              -8.222119e-15   
Biomass IGCC CCS Benzene, pentachloro-                              -1.442291e-12   
Coal IGCC        BOD5, Biological Oxygen Demand                     -2.986431e-04   
                 cement                                             -5.6808

### Only technosphere / inventory flows

In [51]:
techs = ["steel", "concrete", "copper", "aluminium", "electricity", 
         "gas", "diesel", "petrol", "freight", "cement", "heat"]
techdf = fulldf.reset_index()

In [52]:
techdf[techdf["flow"].isin(techs)].set_index(["RMND Tech", "flow"])

amount_nodc     amount_dc  rel. (nodc/dc)  \
RMND Tech        flow                                                      
Biomass CHP      aluminium    1.025982e-06  1.252325e-06        0.819262   
                 cement       8.871915e-05  9.031651e-05        0.982314   
                 concrete     3.501790e-07  3.560818e-07        0.983423   
                 copper       2.266906e-06  2.383243e-06        0.951185   
                 diesel       6.396954e-04  6.425320e-04        0.995585   
                 electricity  5.776253e-03  5.858220e-03        0.986008   
                 freight      2.044194e-02  2.132071e-02        0.958783   
                 gas          4.642754e-05  2.017812e-04        0.230089   
                 heat         3.017027e-02  3.028114e-02        0.996339   
                 petrol       7.964187e-05  7.966587e-05        0.999699   
                 steel        2.178029e-04  2.217810e-04        0.982063   
Biomass IGCC     aluminium    6.445258e-05  1.139406e-04        0.565668   
                 cement       4.292475e-04  6.975558e-04        0.615359   
                 concrete     2.418822e-06  3.526058e-06        0.685985   
                 copper       1.219001e-04  1.479969e-04        0.823666   
                 diesel       3.992283e-03  4.357780e-03        0.916128   
                 electricity  6.928969e-01  7.046066e-01        0.983381   
                 freight      1.339059e-01  2.571119e-01        0.520808   
                 gas          1.351862e-02  3.316039e-02        0.407674   
                 heat         5.747978e-01  5.872738e-01        0.978756   
                 petrol       2.370657e-04  2.414124e-04        0.981994   
                 steel        1.204496e-03  1.878128e-03        0.641328   
Biomass IGCC CCS aluminium    3.237184e-05  6.091050e-05        0.531466   
                 cement       6.826229e-04  8.387773e-04        0.813831   
                 concrete     2.150959e-06  2.791694e-06        0.770485   
                 copper       7.116270e-05  8.637188e-05        0.823910   
                 diesel       8.015953e-03  8.232939e-03        0.973644   
                 electricity  3.975553e-01  4.054817e-01        0.980452   
                 freight      2.925319e-01  3.635461e-01        0.804662   
                 gas          5.214560e-03  1.656090e-02        0.314872   
...                                    ...           ...             ...   
Solar CSP        copper       2.594409e-05  2.650942e-05        0.978675   
                 diesel       6.062202e-04  6.208531e-04        0.976431   
                 electricity  2.172696e-02  2.212561e-02        0.981982   
                 freight      8.595413e-02  8.995692e-02        0.955503   
                 gas          1.367199e-02  1.466749e-02        0.932129   
                 heat         5.256729e-02  5.311665e-02        0.989657   
                 petrol       6.530406e-06  6.676534e-06        0.978113   
                 steel        3.583496e-03  3.604666e-03        0.994127   
Solar PV         aluminium    1.519573e-03  1.525154e-03        0.996340   
                 cement       1.140345e-03  1.239982e-03        0.919646   
                 concrete     1.137490e-06  1.344982e-06        0.845729   
                 copper       3.867768e-04  3.902119e-04        0.991197   
                 diesel       4.857293e-04  5.989866e-04        0.810919   
                 electricity  1.491346e-01  1.519161e-01        0.981690   
                 freight      8.150879e-02  1.123600e-01        0.725425   
                 gas          1.719894e-03  7.673386e-03        0.224138   
                 heat         1.090209e-01  1.130488e-01        0.964371   
                 petrol       8.044166e-06  9.016560e-06        0.892155   
                 steel        1.778951e-03  1.933207e-03        0.920207   
Wind             aluminium    1.189601e-05  1.270345e-05        0.936439   
   

### What about geothermal power plants?

...seem to have quite a large double-counting value (large electricity production?).

In [ ]:
databases

In [ ]:
eidb = Database("ecoinvent_Remind_BAU_2030")

In [ ]:
regions = ei2rmnd.ei_locations_in_remind_region("EUR")
geot = [act for act in eidb if "electricity production" in act["name"]
and "geothermal" in act["name"]
       and act["location"] in regions]

In [ ]:
geot

In [ ]:
lca = LCA({act: 1./len(geot) for act in geot})

In [ ]:
lca.lci()

In [ ]:
bio = Database("biosphere3")

In [ ]:
ant=[act for act in bio if "Antimony" == act["name"]]

In [ ]:
ant_idcs = []
for act in ant:
    try:
        ant_idcs.append(lca.biosphere_dict[act.key])
    except KeyError as e:
        print("Flow not found: {}".format(get_activity(act.key)["name"]))

In [ ]:
ant_contr = lca.inventory[ant_idcs].sum(axis=0).tolist()[0]

In [ ]:
names = [get_activity(key)["name"] for key in lca.activity_dict]

In [ ]:
contrbtrs = pd.Series({"{} | {}".format(names[idx], list(lca.activity_dict.keys())[idx]): ant_contr[idx] for idx in range(len(names))})

In [ ]:
contrbtrs.sort_values()[:20]

In [ ]:
contrbtrs.sum()

The negative impact seems to come from bioenergy power plants.